In [2]:
import os

In [3]:
%pwd

'c:\\Chicken_disease_classififcation_using_deepLearning_withMLDeploymentsINAWS\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Chicken_disease_classififcation_using_deepLearning_withMLDeploymentsINAWS'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifer.constants import *


In [8]:
from src.cnnClassifer.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config  = self.config.data_ingestion

        create_directories([config.root_dir])


        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


     

In [10]:
import os
import urllib.request as request
import zipfile
from cnnClassifer import logger
from src.cnnClassifer.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None"""

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



    

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-06-17 12:22:01,080: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-17 12:22:01,082: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-17 12:22:01,083: INFO: common: created directory at: artifacts]
[2025-06-17 12:22:01,084: INFO: common: created directory at: artifacts/data-ingestion]
[2025-06-17 12:22:06,130: INFO: 60947609: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F1C8:B5609:B0075:216B6F:68510DC2
Accept-Ranges: bytes
Date: Tue, 17 Jun 2025 06:52:01 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10223-MAA
X-Cache: HIT
X